# LAB: SVM: Continuamos con las predicciones sobre datos de cáncer de mama

## 1. Introducción

Continuamos con la tarea encarada en otros labs de clasificación de predecir el diagnóstico de cáncer de mama a partir de las características de las células.


* class_t es la variable target

* el resto son variables con valores normalizados de 1 a 10


[Aquí](https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names) pueden encontrar más información sobre el dataset.

**Nota:** se eliminaron del dataset original 16 casos con valores perdidos en algunos campos.

In [37]:
# Importamos las librerías a utilizar
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [7]:
### Importamos los datos

df = pd.read_csv('breast-cancer.csv', header = None)
df.columns = ['ID', 'clump_Thickness', 'unif_cell_size', 'unif_cell_shape', 'adhesion', 'epith_cell_Size', 'bare_nuclei',
              'bland_chromatin ','norm_nucleoli', 'mitoses', 'class_t']
df.sample(3)

,ID,clump_Thickness,unif_cell_size,unif_cell_shape,adhesion,epith_cell_Size,bare_nuclei,bland_chromatin,norm_nucleoli,mitoses,class_t
168,1200892,8,6,5,4,3,10,6,1,1,4
145,1187805,8,8,7,4,10,10,7,8,7,4
58,1115282,5,3,5,5,3,3,4,10,1,4


## 2. Workflow de clasificación

La idea es implementar el modelo sobre el dataset utilizando cross validation para hacer una búsqueda sobre los hiperparámetros. 
Para tener tanto una estimación de los hiperparámetros óptimos como una idea de la performance sobre el modelo sobre datos nuevos vamos a completar el siguiente workflow:

<img src='worflowtt.png'></img>

Realicen el workflow teniendo en cuenta las siguientes condiciones:

1) Hacer un split train/test incial dejando 75% y 25% de los datos respectivamente

2) A la hora de explorar los hiperparámetros probar los valores de C = [0.1, 0.2, 0.3] y probar con los kernels de tipo radial y lineal



In [8]:
X = df[['clump_Thickness', 'unif_cell_size', 'unif_cell_shape', 'adhesion', 'epith_cell_Size', 'bare_nuclei',
              'bland_chromatin ','norm_nucleoli', 'mitoses']]

In [9]:
y = df['class_t']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=53)

(512, 9) (512,)
(171, 9) (171,)


In [31]:
scores = []
for c in [0.1,0.2,0.3]:
    for krn in ['linear','rbf']:
        score = {}
        score['kernel'] = krn
        score['c'] = c
        model = SVC(C=c,kernel=krn)
        cv_scores = cross_val_score(model,X,y,cv=5)
        score['mean'] = np.mean(cv_scores)
        score['std'] = np.std(cv_scores)
        scores.append(score)

In [32]:
df_scores = pd.DataFrame(scores)

In [33]:
df_scores

,c,kernel,mean,std
0,0.1,linear,0.967840,0.015675
1,0.1,rbf,0.937140,0.032750
2,0.2,linear,0.966380,0.014976
3,0.2,rbf,0.940059,0.029615
4,0.3,linear,0.966380,0.014976
5,0.3,rbf,0.944461,0.031104


In [34]:
best_model = SVC(C=0.1,kernel='linear')

In [35]:
best_model.fit(X_train,y_train)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [36]:
y_pred = best_model.predict(X_test)

In [38]:
accuracy_score(y_test,y_pred)

0.96491228070175439